In [ ]:
# default_exp models.tagnn_pp

# TAGNN++
> [Mitheran et. al. Improved Representation Learning for Session-based Recommendation. arXiv, 2021.](https://arxiv.org/abs/2107.01516v2)

TAGNN models item interactions with GNN, and both local and global user interactions with  a Transformer.

<img src='https://raw.githubusercontent.com/RecoHut-Projects/sessrec-gnn/main/report/images/img7.png'>

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# #export
import math
import datetime
import numpy as np
from collections import Iterable

import torch
from torch import nn, optim
import torch.nn.functional as F

In [ ]:
#exporti
def unitwise_norm(x: torch.Tensor):
    if x.ndim <= 1:
        dim = 0
        keepdim = False
    elif x.ndim in [2, 3]:
        dim = 0
        keepdim = True
    elif x.ndim == 4:
        dim = [1, 2, 3]
        keepdim = True
    else:
        raise ValueError('Wrong dimensions of x')

    return torch.sum(x**2, dim=dim, keepdim=keepdim) ** 0.5

In [ ]:
#exporti
class AGC(optim.Optimizer):
    
    """Generic implementation of the Adaptive Gradient Clipping
    Args:
      params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
      optim (torch.optim.Optimizer): Optimizer with base class optim.Optimizer
      clipping (float, optional): clipping value (default: 1e-3)
      eps (float, optional): eps (default: 1e-3)
      model (torch.nn.Module, optional): The original model
      ignore_agc (str, Iterable, optional): Layers for AGC to ignore
    """

    def __init__(self, params, optim: optim.Optimizer, clipping: float = 1e-2, eps: float = 1e-3, model=None, ignore_agc= ['']):
        if clipping < 0.0:
            raise ValueError("Invalid clipping value: {}".format(clipping))
        if eps < 0.0:
            raise ValueError("Invalid eps value: {}".format(eps))

        self.optim = optim

        defaults = dict(clipping=clipping, eps=eps)
        defaults = {**defaults, **optim.defaults}

        if not isinstance(ignore_agc, Iterable):
            ignore_agc = [ignore_agc]

        if model is not None:
            assert ignore_agc not in [
                None, []], "Specify args ignore_agc to ignore fc-like(or other) layers"
            names = [name for name, module in model.named_modules()]

            for module_name in ignore_agc:
                if module_name not in names:
                    raise ModuleNotFoundError(
                        "Module name {} not found in the model".format(module_name))
            parameters = [{"params": module.parameters()} for name,
                          module in model.named_modules() if name not in ignore_agc]

        super(AGC, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                param_norm = torch.max(unitwise_norm(
                    p.detach()), torch.tensor(group['eps']).to(p.device))
                grad_norm = unitwise_norm(p.grad.detach())
                max_norm = param_norm * group['clipping']

                trigger = grad_norm < max_norm

                clipped_grad = p.grad * \
                    (max_norm / torch.max(grad_norm,
                                          torch.tensor(1e-6).to(grad_norm.device)))
                p.grad.data.copy_(torch.where(trigger, clipped_grad, p.grad))

        return self.optim.step(closure)

In [ ]:
#export
class Attention_GNN(nn.Module):
    def __init__(self, hidden_size, step=1):
        super(Attention_GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        self.w_ih = nn.Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.w_hh = nn.Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.b_ih = nn.Parameter(torch.Tensor(self.gate_size))
        self.b_hh = nn.Parameter(torch.Tensor(self.gate_size))
        self.b_iah = nn.Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = nn.Parameter(torch.Tensor(self.hidden_size))

        self.linear_edge_in = nn.Linear(
            self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = nn.Linear(
            self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = nn.Linear(
            self.hidden_size, self.hidden_size, bias=True)

    def GNNCell(self, A, hidden):
        input_in = torch.matmul(A[:, :, :A.shape[1]],
                                self.linear_edge_in(hidden)) + self.b_iah

        input_out = torch.matmul(
            A[:, :, A.shape[1]: 2 * A.shape[1]], self.linear_edge_out(hidden)) + self.b_oah

        inputs = torch.cat([input_in, input_out], 2)
        gi = F.linear(inputs, self.w_ih, self.b_ih)
        gh = F.linear(hidden, self.w_hh, self.b_hh)
        i_r, i_i, i_n = gi.chunk(3, 2)
        h_r, h_i, h_n = gh.chunk(3, 2)
        resetgate = torch.sigmoid(i_r + h_r)
        inputgate = torch.sigmoid(i_i + h_i)
        newgate = torch.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def forward(self, A, hidden):
        for i in range(self.step):
            hidden = self.GNNCell(A, hidden)
        return hidden

In [ ]:
#export
class TAGNN_PP(nn.Module):
    def __init__(self, opt):
        super(TAGNN_PP, self).__init__()
        self.hidden_size = args.hiddenSize
        self.n_node = opt.n_node
        self.batch_size = args.batchSize
        self.nonhybrid = args.nonhybrid
        self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.tagnn = Attention_GNN(self.hidden_size, step=args.step)

        self.layer_norm1 = nn.LayerNorm(self.hidden_size)
        self.attn = nn.MultiheadAttention(
            embed_dim=self.hidden_size, num_heads=2, dropout=0.1)

        self.linear_one = nn.Linear(
            self.hidden_size, self.hidden_size, bias=True)

        self.linear_two = nn.Linear(
            self.hidden_size, self.hidden_size, bias=True)

        self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_transform = nn.Linear(
            self.hidden_size * 2, self.hidden_size, bias=True)
        self.linear_t = nn.Linear(
            self.hidden_size, self.hidden_size, bias=False)  # target attention
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(
            self.parameters(), lr=args.lr, weight_decay=args.l2)
        self.agc_optimizer = AGC(self.parameters(), self.optimizer, model=self)
        self.scheduler = torch.optim.lr_scheduler.StepLR(
            self.optimizer, step_size=args.lr_dc_step, gamma=args.lr_dc)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def compute_scores(self, hidden, mask):
        ht = hidden[torch.arange(mask.shape[0]).long(), torch.sum(
            mask, 1) - 1]  # batch_size x latent_size
        # batch_size x 1 x latent_size
        q1 = self.linear_one(ht).view(ht.shape[0], 1, ht.shape[1])
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        # batch_size x seq_length x 1
        alpha = self.linear_three(torch.sigmoid(q1 + q2))
        alpha = F.softmax(alpha, 1)  # batch_size x seq_length x 1
        # batch_size x latent_size
        a = torch.sum(alpha * hidden *
                      mask.view(mask.shape[0], -1, 1).float(), 1)

        if not self.nonhybrid:
            a = self.linear_transform(torch.cat([a, ht], 1))
        b = self.embedding.weight[1:]  # n_nodes x latent_size

        # batch_size x seq_length x latent_size
        hidden = hidden * mask.view(mask.shape[0], -1, 1).float()
        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size
        # batch_size x n_nodes x seq_length
        beta = F.softmax(b @ qt.transpose(1, 2), -1)
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = a.view(ht.shape[0], 1, ht.shape[1])  # batch_size x 1 x latent_size
        a = a + target  # batch_size x n_nodes x latent_size
        scores = torch.sum(a * b, -1)  # batch_size x n_nodes
        return scores

    @staticmethod
    def get_mask(seq_len):
        return torch.from_numpy(np.triu(np.ones((seq_len, seq_len)), k=1).astype('bool'))#.to('cuda')

    def forward(self, inputs, A):
        hidden = self.embedding(inputs)
        hidden = self.tagnn(A, hidden)
        hidden = hidden.permute(1, 0, 2)

        skip = self.layer_norm1(hidden)
        hidden, attn_w = self.attn(
            hidden, hidden, hidden, attn_mask=self.get_mask(hidden.shape[0]))
        hidden = hidden+skip
        hidden = hidden.permute(1, 0, 2)

        return hidden

## Training a session-based recommender using TAGNN++

In [ ]:
class Args():
    dataset = 'sample'
    batchSize = 100 # input batch size
    hiddenSize = 100 # hidden state size
    epoch = 5 # the number of epochs to train for
    lr = 0.001 # learning rate')  # [0.001, 0.0005, 0.000
    lr_dc = 0.1 # learning rate decay rate
    lr_dc_step = 3 # the number of steps after which the learning rate decay
    l2 = 1e-5 # l2 penalty')  # [0.001, 0.0005, 0.0001, 0.00005, 0.0000
    step = 1 # gnn propogation steps
    patience = 10 # the number of epoch to wait before early stop 
    nonhybrid = True # only use the global preference to predict
    validation = True # validation
    valid_portion = 0.1 # split the portion of training set as validation set
    n_node = 310

args = Args()

In [ ]:
def to_cuda(input_variable):
    if torch.cuda.is_available():
        return input_variable.cuda()
    else:
        return input_variable


def to_cpu(input_variable):
    if torch.cuda.is_available():
        return input_variable.cpu()
    else:
        return input_variable


def forward(model, i, data):
    alias_inputs, A, items, mask, targets = data.get_slice(i)
    alias_inputs = to_cuda(torch.Tensor(alias_inputs).long())
    items = to_cuda(torch.Tensor(items).long())
    A = to_cuda(torch.Tensor(A).float())
    mask = to_cuda(torch.Tensor(mask).long())
    hidden = model(items, A)

    def get(i): return hidden[i][alias_inputs[i]]
    seq_hidden = torch.stack([get(i)
                             for i in torch.arange(len(alias_inputs)).long()])

    return targets, model.compute_scores(seq_hidden, mask)


def train_test(model, train_data, test_data):
    model.scheduler.step()
    print('Start training: ', datetime.datetime.now())
    model.train()
    total_loss = 0.0
    slices = train_data.generate_batch(model.batch_size)

    from tqdm.notebook import tqdm
    for i, j in tqdm(zip(slices, np.arange(len(slices))), total=len(slices)):
        model.optimizer.zero_grad()
        targets, scores = forward(model, i, train_data)
        targets = to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        total_loss += loss.item()

        if j % int(len(slices) / 5 + 1) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), loss.item()))

    print('\tLoss Value:\t%.3f' % total_loss)
    print('Start Prediction: ', datetime.datetime.now())

    model.eval()
    hit, mrr = [], []
    slices = test_data.generate_batch(model.batch_size)

    for i in slices:
        targets, scores = forward(model, i, test_data)
        sub_scores = scores.topk(20)[1]
        sub_scores = to_cpu(sub_scores).detach().numpy()

        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr


def get_pos(seq_len):
    return torch.arange(seq_len).unsqueeze(0)


def str2bool(v):
    return v.lower() in ('true')


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)

In [ ]:
import os
import pickle
import time
from torch.utils.tensorboard import SummaryWriter

from recohut.datasets.session import SampleSessionDataset, GraphData

import warnings
warnings.filterwarnings('ignore')

model_save_dir = 'saved/'
log_dir='saved/logs'
writer = SummaryWriter(log_dir=log_dir)
os.makedirs(log_dir, exist_ok=True)

_ = SampleSessionDataset('./session_ds')
train_data = pickle.load(open('./session_ds/processed/train.txt', 'rb'))

if args.validation:
    train_data, valid_data = split_validation(train_data, args.valid_portion)
    test_data = valid_data
else:
    test_data = pickle.load(open('./session_ds/processed/test.txt', 'rb'))

train_data = GraphData(train_data, shuffle=True)
test_data = GraphData(test_data, shuffle=False)

model = to_cuda(TAGNN_PP(args))

start = time.time()
best_result = [0, 0]
best_epoch = [0, 0]
bad_counter = 0

for epoch in range(args.epoch):
    print('-' * 50)
    print('Epoch: ', epoch)
    hit, mrr = train_test(model, train_data, test_data)
    flag = 0

    # Logging
    writer.add_scalar('epoch/recall', hit, epoch)
    writer.add_scalar('epoch/mrr', mrr, epoch)

    flag = 0

    if hit >= best_result[0]:
        best_result[0] = hit
        best_epoch[0] = epoch
        flag = 1
        torch.save(model, model_save_dir + 'epoch_' +
                    str(epoch) + '_recall_' + str(hit) + '_.pt')
    if mrr >= best_result[1]:
        best_result[1] = mrr
        best_epoch[1] = epoch
        flag = 1
        torch.save(model, model_save_dir + 'epoch_' +
                    str(epoch) + '_mrr_' + str(mrr) + '_.pt')

    print('Best Result:')
    print('\tRecall@20:\t%.4f\tMRR@20:\t%.4f\tEpoch:\t%d,\t%d' %
            (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))

    bad_counter += 1 - flag

    if bad_counter >= args.patience:
        break

print('-' * 50)
end = time.time()
print("Running time: %f seconds" % (end - start))

--------------------------------------------------
Epoch:  0
Start training:  2021-12-23 11:37:30.078500


  0%|          | 0/11 [00:00<?, ?it/s]

[0/11] Loss: 5.7335
[3/11] Loss: 5.7263
[6/11] Loss: 5.7291
[9/11] Loss: 5.7135
	Loss Value:	62.968
Start Prediction:  2021-12-23 11:37:31.807639
Best Result:
	Recall@20:	17.3554	MRR@20:	5.2499	Epoch:	0,	0
--------------------------------------------------
Epoch:  1
Start training:  2021-12-23 11:37:31.935884


  0%|          | 0/11 [00:00<?, ?it/s]

[0/11] Loss: 5.7028
[3/11] Loss: 5.6784
[6/11] Loss: 5.6666
[9/11] Loss: 5.6629
	Loss Value:	62.498
Start Prediction:  2021-12-23 11:37:33.677782
Best Result:
	Recall@20:	19.8347	MRR@20:	5.2499	Epoch:	1,	0
--------------------------------------------------
Epoch:  2
Start training:  2021-12-23 11:37:33.802865


...

[0/11] Loss: 5.6609
[3/11] Loss: 5.6498
[6/11] Loss: 5.6462
[9/11] Loss: 5.6439
	Loss Value:	62.090
Start Prediction:  2021-12-23 11:37:35.502085
Best Result:
	Recall@20:	20.6612	MRR@20:	5.2499	Epoch:	2,	0
--------------------------------------------------
Epoch:  3
Start training:  2021-12-23 11:37:35.604826


  0%|          | 0/11 [00:00<?, ?it/s]

[0/11] Loss: 5.6461
[3/11] Loss: 5.6324
[6/11] Loss: 5.6430
[9/11] Loss: 5.6417
	Loss Value:	61.997
Start Prediction:  2021-12-23 11:37:37.213415
Best Result:
	Recall@20:	21.4876	MRR@20:	5.2499	Epoch:	3,	0
--------------------------------------------------
Epoch:  4
Start training:  2021-12-23 11:37:37.333273


  0%|          | 0/11 [00:00<?, ?it/s]

[0/11] Loss: 5.6551
[3/11] Loss: 5.6529
[6/11] Loss: 5.5994
[9/11] Loss: 5.6067
	Loss Value:	61.898
Start Prediction:  2021-12-23 11:37:38.855207
Best Result:
	Recall@20:	21.4876	MRR@20:	5.2499	Epoch:	4,	0
--------------------------------------------------
Running time: 8.897622 seconds


---

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-23 11:37:42

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython: 5.5.0
numpy  : 1.19.5
torch  : 1.10.0+cu111

